### Generating the index for Q6

In [1]:
import json
with open('/home/thummala/Downloads/netflix.json') as f:
  netflix = json.load(f)
netflix

{'s1': {'type': 'TV Show',
  'title': '3%',
  'director': '',
  'cast': 'João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi',
  'country': 'Brazil',
  'date_added': 'August 14, 2020',
  'release_year': 2020,
  'rating': 'TV-MA',
  'duration': '4 Seasons',
  'listed_in': 'International TV Shows, TV Dramas, TV Sci-Fi & Fantasy',
  'description': 'In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor.'},
 's2': {'type': 'Movie',
  'title': '7:19',
  'director': 'Jorge Michel Grau',
  'cast': 'Demián Bichir, Héctor Bonilla, Oscar Serrano, Azalia Ortiz, Octavio Michel, Carmen Beato',
  'country': 'Mexico',
  'date_added': 'December 23, 2016',
  'release_year': 2016,
  'rating': 'TV-MA',
  'duration': '93 min',
  'listed_in': 'Dramas, International Movies',
  'description': 'After a devastatin

In [2]:
len(netflix.keys())

7787

In [3]:
netflix['s1']

{'type': 'TV Show',
 'title': '3%',
 'director': '',
 'cast': 'João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi',
 'country': 'Brazil',
 'date_added': 'August 14, 2020',
 'release_year': 2020,
 'rating': 'TV-MA',
 'duration': '4 Seasons',
 'listed_in': 'International TV Shows, TV Dramas, TV Sci-Fi & Fantasy',
 'description': 'In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor.'}

In [4]:
from elasticsearch import Elasticsearch

/home/thummala/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
es=Elasticsearch()
keys=list(netflix.keys())

In [46]:
print(len(keys))
print(len(netflix[keys[0]].keys()))

7787
11


In [47]:
netflix[keys[0]]

{'type': 'TV Show',
 'title': '3%',
 'director': '',
 'cast': 'João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi',
 'country': 'Brazil',
 'date_added': 'August 14, 2020',
 'release_year': 2020,
 'rating': 'TV-MA',
 'duration': '4 Seasons',
 'listed_in': 'International TV Shows, TV Dramas, TV Sci-Fi & Fantasy',
 'description': 'In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor.'}

In [96]:
ratings=[]
for key in keys:
    ratings.append(netflix[key]['rating'])
ratings=set(ratings)
ratings

{'',
 'G',
 'NC-17',
 'NR',
 'PG',
 'PG-13',
 'R',
 'TV-14',
 'TV-G',
 'TV-MA',
 'TV-PG',
 'TV-Y',
 'TV-Y7',
 'TV-Y7-FV',
 'UR'}

## Generating the index

In [57]:
analyzer={
  "settings": {
    "index": {
      "analysis": {
        "filter": {},
        "analyzer": {
          "keyword_analyzer": {
                "filter": [
                  "lowercase",
                  "asciifolding",
                  "trim"
                ],
                "char_filter": [],
                "type": "custom",
                "tokenizer": "keyword"
            },
            "edge_ngram_analyzer": {
                "filter": [
                  "lowercase"
                ],
                "tokenizer": "edge_ngram_tokenizer"
            },
            "edge_ngram_search_analyzer": {
                "tokenizer": "lowercase"
            },
            "ngram_analyser" : {
                "filter": [
                  "lowercase"
                ],
                "tokenizer": "ngram_analyser"
            },
            "ngram_search_analyser" : {
                "tokenizer" : "lowercase"
            }
        },
        "tokenizer": {
          "edge_ngram_tokenizer": {
            "type": "edge_ngram",
            "min_gram": 2,
            "max_gram": 5,
            "token_chars": [
              "letter"
            ]
          },
          "ngram_analyser" :{
              "type": "ngram",
              "min_gram": 5,
              "max_gram": 5,
              "token_chars": [
                "letter"
              ]
            }
        }
      }
    }
  },
  "mappings": {
      "properties": {
        "title": {
          "type": "text",
          "fields": {
            "keywordstring": {
              "type": "text",
              "analyzer": "keyword_analyzer"
            },
            "edgengram": {
              "type": "text",
              "analyzer": "edge_ngram_analyzer",
              "search_analyzer": "edge_ngram_search_analyzer"
            },
            "completion": {
              "type": "completion"
            },
            "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
            }  
          },
          "analyzer": "standard"
        },
      "type" : {
            "type" : "text",
            "fields" : {
                "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
                }
            },
            "analyzer": "standard"
        },
      "director" : {
            "type" : "text",
            "fields" : {
                "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
                }
            },
            "analyzer": "standard"
        },
      "cast" : {
            "type" : "text",
            "fields" : {
                "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
                }
            },
            "analyzer": "standard"
        },
      "country" : {
            "type" : "text",
            "fields" : {
                "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
                }
            },
            "analyzer": "standard"
        },
      "date_added" : {
            "type" : "text",
            "fields" : {
                "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
                }
            },
            "analyzer": "standard"
        },
      "release_year" : {
            "type" : "text",
            "fields" : {
                "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
                }
            },
            "analyzer": "standard"
        },
      "rating" : {
            "type" : "text",
            "fields" : {
                "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
                }
            },
            "analyzer": "standard"
        },
      "duration" : {
            "type" : "text",
            "fields" : {
                "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
                }
            },
            "analyzer": "standard"
        },
      "listed_in" : {
            "type" : "text",
            "fields" : {
                "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
                }
            },
            "analyzer": "standard"
        },
      "description" : {
            "type" : "text",
            "fields" : {
                "ngram" : {
                    "type" : "text",
                    "analyzer" : "ngram_analyser"
                },
                "keywordstring": {
                      "type": "text",
                      "analyzer": "keyword_analyzer"
              }
            },
            "analyzer": "standard"
        },
      }
    }
  }

In [58]:
#es.indices.delete(index='netflix_final', ignore=[400, 404])
res=es.indices.create('netflix_final',body=analyzer,request_timeout=30)
#es.indices.exists('netflix-testing2')


In [59]:
for i in range(len(keys)):
    doc=netflix[keys[i]]
    res = es.index(index="netflix_final", id=i, body=doc, request_timeout=120)
    if res['result'] != 'created':
        print(res['result'])

### Generating the index for part b of Q6

In [ ]:
analyzer={
  "settings": {
    "index": {
      "analysis": {
        "filter": {},
        "analyzer": {
          "keyword_analyzer": {
            "filter": [
              "lowercase",
              "asciifolding",
              "trim"
            ],
            "char_filter": [],
            "type": "custom",
            "tokenizer": "keyword"
          },
          "edge_ngram_analyzer": {
            "filter": [
              "lowercase"
            ],
            "tokenizer": "edge_ngram_tokenizer"
          },
          "edge_ngram_search_analyzer": {
            "tokenizer": "lowercase"
          }
        },
        "tokenizer": {
          "edge_ngram_tokenizer": {
            "type": "edge_ngram",
            "min_gram": 2,
            "max_gram": 5,
            "token_chars": [
              "letter"
            ]
          }
        }
      }
    }
  },
  "mappings": {
      "properties": {
        "title": {
          "type": "text",
          "fields": {
            "keywordstring": {
              "type": "text",
              "analyzer": "keyword_analyzer"
            },
            "edgengram": {
              "type": "text",
              "analyzer": "edge_ngram_analyzer",
              "search_analyzer": "edge_ngram_search_analyzer"
            },
            "completion": {
              "type": "completion"
            }
          },
          "analyzer": "standard"
        },
          "description": {
              "type":"text",
              "fields":{
            "keywordstring": {
              "type": "text",
              "analyzer": "keyword_analyzer"
              }
          },
          "analyzer": "standard"    
      }
    }
  }
}
####################################################

In [ ]:
res=es.indices.create('netflix-testing2',body=analyzer,request_timeout=30)

In [ ]:
for i in range(len(keys)):
    doc=netflix[keys[i]]
    res = es.index(index="netflix-testing2", id=i, body=doc, request_timeout=120)
    if res['result'] != 'created':
        print(res['result'])

### Ignore this section -- It is for veryfying the search queries

In [50]:
def endp_adults(text):
    q={
      "query": {
        "match": {
          "title.edgengram": text
        }
      }
    }

    res=es.search(index='netflix_final',body=q)
    ret_list=[]
    for index,item in enumerate(res['hits']['hits']):
        ret_list.append({'title':item['_source']['title'],'director':[item['_source']['director']],'rating':item['_source']['rating']})
    return ret_list
def endp_childs(text):
    
    q = {
      "query": {
        "bool" : {
          "should" : [
                    {"match": {
              "title.edgengram": text
            }}
          ],
            "must_not":[
                {"terms":{"rating.keyword":["R","TV-PG",'PG','PG-13','NC-17','']}}
            ],

          "boost" : 1.0
        }
      }
    }


    res=es.search(index='netflix_final',body=q)
    ret_list=[]
    for index,item in enumerate(res['hits']['hits']):
        ret_list.append({'title':item['_source']['title'],'director':[item['_source']['director']],'rating':item['_source']['rating']})
    return ret_list

In [5]:
res=endp_adults('fight')

In [6]:
res

[{'title': 'FIGHTWORLD', 'director': ['Padraic McKinley'], 'rating': 'TV-MA'},
 {'title': 'The Fighter', 'director': ['David O. Russell'], 'rating': 'R'},
 {'title': 'Pahuyut Fighting Beat',
  'director': ['Piti Jaturaphat'],
  'rating': 'TV-MA'},
 {'title': 'The Big Flower Fight', 'director': [''], 'rating': 'TV-PG'},
 {'title': 'You Can’t Fight Christmas',
  'director': ['Kenny Young'],
  'rating': 'TV-PG'},
 {'title': 'Craig Ferguson: Tickle Fight',
  'director': ['Jay Chapman'],
  'rating': 'TV-MA'},
 {'title': 'Stretch Armstrong & the Flex Fighters',
  'director': [''],
  'rating': 'TV-Y7'},
 {'title': 'Why We Fight: The Battle of Russia',
  'director': ['Frank Capra, Anatole Litvak'],
  'rating': 'TV-PG'},
 {'title': "100 Years: One Woman's Fight for Justice",
  'director': ['Melinda Janko'],
  'rating': 'TV-14'},
 {'title': "Winter on Fire: Ukraine's Fight for Freedom",
  'director': ['Evgeny Afineevsky'],
  'rating': 'TV-MA'}]

In [7]:
res=endp_childs('fight')

In [8]:
res

[{'title': 'FIGHTWORLD', 'director': ['Padraic McKinley'], 'rating': 'TV-MA'},
 {'title': 'Pahuyut Fighting Beat',
  'director': ['Piti Jaturaphat'],
  'rating': 'TV-MA'},
 {'title': 'Craig Ferguson: Tickle Fight',
  'director': ['Jay Chapman'],
  'rating': 'TV-MA'},
 {'title': 'Stretch Armstrong & the Flex Fighters',
  'director': [''],
  'rating': 'TV-Y7'},
 {'title': "100 Years: One Woman's Fight for Justice",
  'director': ['Melinda Janko'],
  'rating': 'TV-14'},
 {'title': "Winter on Fire: Ukraine's Fight for Freedom",
  'director': ['Evgeny Afineevsky'],
  'rating': 'TV-MA'}]

In [60]:
def prefix_match_endpoint(text):
    query_string=text+'.*'
    q={
  "query": {
    "regexp": {
      "description.keywordstring": {
        "value": query_string,
        "flags": "ALL",
        "case_insensitive": True,
        "max_determinized_states": 10000,
        "rewrite": "constant_score"
      }
    }
  }
}
    res=es.search(index='netflix_final',body=q)
    ret_list=[]
    for index,item in enumerate(res['hits']['hits']):
        ret_list.append({'title':item['_source']['title'],'director':[item['_source']['director']],'description':item['_source']['description']})
    return ret_list


In [39]:
results=prefix_match_endpoint('After')
results

[{'title': '7:19',
  'director': ['Jorge Michel Grau'],
  'description': 'After a devastating earthquake hits Mexico City, trapped survivors from all walks of life wait to be rescued while trying desperately to stay alive.'},
 {'title': '122',
  'director': ['Yasir Al Yasiri'],
  'description': 'After an awful accident, a couple admitted to a grisly hospital are separated and must find each other to escape — before death finds them.'},
 {'title': '187',
  'director': ['Kevin Reynolds'],
  'description': 'After one of his high school students attacks him, dedicated teacher Trevor Garfield grows weary of the gang warfare in the New York City school system and moves to California to teach there, thinking it must be a less hostile environment.'},
 {'title': '22-Jul',
  'director': ['Paul Greengrass'],
  'description': 'After devastating terror attacks in Norway, a young survivor, grieving families and the country rally for justice and healing. Based on a true story.'},
 {'title': '\u200b\u

In [61]:
results=prefix_match_endpoint('After')
results

[{'title': '1 Mile to You',
  'director': ['Leif Tilden'],
  'description': 'After escaping the bus accident that killed his girlfriend, a high school student channels his grief into running, with the help of a new coach.'},
 {'title': '10 Days in Sun City',
  'director': ['Adze Ugah'],
  'description': 'After his girlfriend wins the Miss Nigeria pageant, a young man faces unexpected competition of his own when he joins her on a campaign in South Africa.'},
 {'title': '122',
  'director': ['Yasir Al Yasiri'],
  'description': 'After an awful accident, a couple admitted to a grisly hospital are separated and must find each other to escape — before death finds them.'},
 {'title': '187',
  'director': ['Kevin Reynolds'],
  'description': 'After one of his high school students attacks him, dedicated teacher Trevor Garfield grows weary of the gang warfare in the New York City school system and moves to California to teach there, thinking it must be a less hostile environment.'},
 {'title': 

In [62]:
def match_phrase(field,text):
    ret_list=[]
    q={
  "query": {
    "bool": {
      "must": [
         {"match_phrase": {field:text}}
      ]
    }
 }
}
    res=es.search(index='netflix_final',body=q)
    ret_list=[]
    for index,item in enumerate(res['hits']['hits']):
        ret_list.append({'title':item['_source']['title'],'director':[item['_source']['director']],'description':item['_source']['description']})
    return ret_list

    

In [65]:
results=match_phrase('director','Jastis Arimbab')

In [66]:
results

[]